In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


### Reading the dataset 

In [5]:
fb1 = pd.read_csv("/kaggle/input/smartphone-decimeter-2023/sdc2023/train/2023-09-07-22-48-us-ca-routebc2/pixel4xl/device_gnss.csv")
df1 = pd.read_csv("/kaggle/input/smartphone-decimeter-2023/sdc2023/train/2023-09-07-22-48-us-ca-routebc2/pixel4xl/ground_truth.csv")

In [7]:
fb = pd.concat([fb1, df1], axis=1)

In [10]:
for col, val in zip(fb.columns, fb.isna().sum()):
   print(f"{col}: {val}")

MessageType: 0
utcTimeMillis: 0
TimeNanos: 0
LeapSecond: 0
TimeUncertaintyNanos: 77455
FullBiasNanos: 0
BiasNanos: 0
BiasUncertaintyNanos: 0
DriftNanosPerSecond: 0
DriftUncertaintyNanosPerSecond: 0
HardwareClockDiscontinuityCount: 0
Svid: 0
TimeOffsetNanos: 0
State: 0
ReceivedSvTimeNanos: 0
ReceivedSvTimeUncertaintyNanos: 0
Cn0DbHz: 0
PseudorangeRateMetersPerSecond: 0
PseudorangeRateUncertaintyMetersPerSecond: 0
AccumulatedDeltaRangeState: 0
AccumulatedDeltaRangeMeters: 0
AccumulatedDeltaRangeUncertaintyMeters: 0
CarrierFrequencyHz: 0
CarrierCycles: 77455
CarrierPhase: 77455
CarrierPhaseUncertainty: 77455
MultipathIndicator: 0
SnrInDb: 77455
ConstellationType: 0
AgcDb: 77455
BasebandCn0DbHz: 77455
FullInterSignalBiasNanos: 77455
FullInterSignalBiasUncertaintyNanos: 77455
SatelliteInterSignalBiasNanos: 77455
SatelliteInterSignalBiasUncertaintyNanos: 77455
CodeType: 77455
ChipsetElapsedRealtimeNanos: 0
ArrivalTimeNanosSinceGpsEpoch: 0
RawPseudorangeMeters: 0
RawPseudorangeUncertaintyMete

In [12]:
fb.columns

Index(['MessageType', 'utcTimeMillis', 'TimeNanos', 'LeapSecond',
       'TimeUncertaintyNanos', 'FullBiasNanos', 'BiasNanos',
       'BiasUncertaintyNanos', 'DriftNanosPerSecond',
       'DriftUncertaintyNanosPerSecond', 'HardwareClockDiscontinuityCount',
       'Svid', 'TimeOffsetNanos', 'State', 'ReceivedSvTimeNanos',
       'ReceivedSvTimeUncertaintyNanos', 'Cn0DbHz',
       'PseudorangeRateMetersPerSecond',
       'PseudorangeRateUncertaintyMetersPerSecond',
       'AccumulatedDeltaRangeState', 'AccumulatedDeltaRangeMeters',
       'AccumulatedDeltaRangeUncertaintyMeters', 'CarrierFrequencyHz',
       'CarrierCycles', 'CarrierPhase', 'CarrierPhaseUncertainty',
       'MultipathIndicator', 'SnrInDb', 'ConstellationType', 'AgcDb',
       'BasebandCn0DbHz', 'FullInterSignalBiasNanos',
       'FullInterSignalBiasUncertaintyNanos', 'SatelliteInterSignalBiasNanos',
       'SatelliteInterSignalBiasUncertaintyNanos', 'CodeType',
       'ChipsetElapsedRealtimeNanos', 'ArrivalTimeNanosSince

In [9]:
fb = fb[~fb["SvVelocityXEcefMetersPerSecond"].isna()]


In [49]:
fb = fb[~fb["ArrivalTimeNanosSinceGpsEpoch"].isna()]

In [54]:
fb = fb.drop(["CarrierCycles","CarrierPhaseUncertainty","SnrInDb","AgcDb","BasebandCn0DbHz","FullInterSignalBiasNanos","FullInterSignalBiasUncertaintyNanos","SatelliteInterSignalBiasNanos","SatelliteInterSignalBiasUncertaintyNanos","CodeType","TimeUncertaintyNanos"])

KeyError: "['CarrierCycles', 'CarrierPhaseUncertainty', 'SnrInDb', 'AgcDb', 'BasebandCn0DbHz', 'FullInterSignalBiasNanos', 'FullInterSignalBiasUncertaintyNanos', 'SatelliteInterSignalBiasNanos', 'SatelliteInterSignalBiasUncertaintyNanos', 'CodeType', 'TimeUncertaintyNanos'] not found in axis"

In [32]:
fb = fb.drop(columns=["TimeUncertaintyNanos"])

In [42]:
fb["State"]

0        16399
1        16399
2        16399
3        16399
4        16399
         ...  
78221    81935
78222    81935
78223    81935
78224    81935
78225    81935
Name: State, Length: 77455, dtype: int64

In [38]:
num_of_numerical_values=fb.select_dtypes(np.number).shape[1]
print(f"Numbers of colums with numerical values - {num_of_numerical_values}")
print(f"number of categorical variable- {fb.shape[1]-num_of_numerical_values}")

Numbers of colums with numerical values - 44
number of categorical variable- 2


In [1]:
Unix_time_Mils = ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters',
       'WlsPositionZEcefMeters']
X = fb[[Unix_time_Mils]]
y = fb['UnixTimeMillis']  

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Making predictions on the testing set
predictions = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

In [ ]:
LongitudeDegrees_features = ['SvPositionYEcefMeters', 'SvPositionZEcefMeters', 'SvElevationDegrees']
X = fb[[LongitudeDegrees_features]]
y = fb['LongitudeDegrees'] 

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Making predictions on the testing set
predictions = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

In [30]:
from sklearn.impute import SimpleImputer

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Create and train a Linear Regression model
model = LinearRegression()
model.fit(X_train_imputed, y_train)

# Make predictions on the testing set
predictions = model.predict(X_test_imputed)

# Create a new column in the dataframe with the predictions
fb['predictions'] = model.predict(imputer.transform(X))

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


ValueError: Input y contains NaN.

In [31]:
print(y_train.unique())
print(y_train.isnull().sum())

[         nan -121.9124547 -121.9020743 ... -121.9345987 -121.9252376
 -121.928164 ]
60341
